In [97]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [183]:
CLIENT_ID = '0VHQ2UD5XSSLMMMYGDDNEE25HDVXSXE3PHO5K0ETG5ARETOJ' # your Foursquare ID
CLIENT_SECRET = 'G4DE4JLYC55DCHBGXEXDVYEAAUKSWBWAKQCYFD2D1AMVRN5J' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0VHQ2UD5XSSLMMMYGDDNEE25HDVXSXE3PHO5K0ETG5ARETOJ
CLIENT_SECRET:G4DE4JLYC55DCHBGXEXDVYEAAUKSWBWAKQCYFD2D1AMVRN5J


In [184]:
address = 'Paris'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [185]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=0VHQ2UD5XSSLMMMYGDDNEE25HDVXSXE3PHO5K0ETG5ARETOJ&client_secret=G4DE4JLYC55DCHBGXEXDVYEAAUKSWBWAKQCYFD2D1AMVRN5J&ll=48.8566101,2.3514992&v=20180604&radius=500&limit=30'

In [186]:
results = requests.get(url).json()
'There are {} popular places in Paris.'.format(len(results['response']['groups'][0]['items']))

'There are 30 popular places in Paris.'

In [187]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4bf41231e5eba59334341f90-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d164941735',
    'name': 'Plaza',
    'pluralName': 'Plazas',
    'primary': True,
    'shortName': 'Plaza'}],
  'id': '4bf41231e5eba59334341f90',
  'location': {'address': "Place de l'Hôtel de Ville",
   'cc': 'FR',
   'city': 'Paris',
   'country': 'France',
   'distance': 60,
   'formattedAddress': ["Place de l'Hôtel de Ville", '75004 Paris', 'France'],
   'lat': 48.85713761176484,
   'lng': 2.3516931321499746,
   'postalCode': '75004',
   'state': 'Île-de-France'},
  'name': "Place de l'Hôtel de Ville – Esplanade de la Libération",
  'photos': {'count': 0, 'groups': []}}}

In [188]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [189]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,FR,Paris,France,NaN,60,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,48.857138,2.351693,75004,Île-de-France,4bf41231e5eba59334341f90
1,Häagen-Dazs,Ice Cream Shop,1 rue d'Arcole,FR,Paris,France,NaN,218,"[1 rue d'Arcole, 75004 Paris, France]","[{'label': 'display', 'lat': 48.85487262126511...",48.854873,2.350125,75004,Île-de-France,4cb2fc20db32f04d6da9d34d
2,LUSH,Cosmetics Shop,3 rue du Renard,FR,Paris,France,NaN,131,"[3 rue du Renard, 75004 Paris, France]","[{'label': 'display', 'lat': 48.85779127633849...",48.857791,2.351622,75004,Île-de-France,4d628e33e4fe548152b8979e
3,L'Alsacien,Alsatian Restaurant,6 rue Saint-Bon,FR,Paris,France,NaN,202,"[6 rue Saint-Bon, 75004 Paris, France]","[{'label': 'display', 'lat': 48.85827502747949...",48.858275,2.350381,75004,Île-de-France,54351b59498e123ee4afc7ff
4,BHV Marais,Department Store,52 rue de Rivoli,FR,Paris,France,Rue de Rivoli,171,"[52 rue de Rivoli (Rue de Rivoli), 75004 Paris...","[{'label': 'display', 'lat': 48.85721020405602...",48.857210,2.353651,75004,Île-de-France,4b4b6a3df964a520b89a26e3


In [190]:
# Get the location data for each borough of Paris
address = '75000'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudezero = location.latitude
longitudezero = location.longitude
print('The geograpical coordinate of 75000 are {}, {}.'.format(latitudezero, longitudezero))

address = '75001'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeone = location.latitude
longitudeone = location.longitude
print('The geograpical coordinate of 75001 are {}, {}.'.format(latitudeone, longitudeone))

address = '75002'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudetwo = location.latitude
longitudetwo = location.longitude
print('The geograpical coordinate of 75002 are {}, {}.'.format(latitudetwo, longitudetwo))

address = '75003'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudethree = location.latitude
longitudethree = location.longitude
print('The geograpical coordinate of 75003 are {}, {}.'.format(latitudethree, longitudethree))

address = '75004'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefour = location.latitude
longitudefour = location.longitude
print('The geograpical coordinate of 75004 are {}, {}.'.format(latitudefour, longitudefour))

address = '75005'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefive = location.latitude
longitudefive = location.longitude
print('The geograpical coordinate of 75005 are {}, {}.'.format(latitudefive, longitudefive))

address = '75006'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudesix = location.latitude
longitudesix = location.longitude
print('The geograpical coordinate of 75006 are {}, {}.'.format(latitudesix, longitudesix))

address = '75007'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeseven = location.latitude
longitudeseven = location.longitude
print('The geograpical coordinate of 75007 are {}, {}.'.format(latitudeseven, longitudeseven))

address = '75008'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeeight = location.latitude
longitudeeight = location.longitude
print('The geograpical coordinate of 75008 are {}, {}.'.format(latitudeeight, longitudeeight))

address = '75009'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudenine = location.latitude
longitudenine = location.longitude
print('The geograpical coordinate of 75009 are {}, {}.'.format(latitudenine, longitudenine))

address = '75010'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeten = location.latitude
longitudeten = location.longitude
print('The geograpical coordinate of 75010 are {}, {}.'.format(latitudeten, longitudeten))

address = '75011'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeeleven = location.latitude
longitudeeleven = location.longitude
print('The geograpical coordinate of 75011 are {}, {}.'.format(latitudeeleven, longitudeeleven))

address = '75012'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudetwelve = location.latitude
longitudetwelve = location.longitude
print('The geograpical coordinate of 75012 are {}, {}.'.format(latitudetwelve, longitudetwelve))

address = '75013'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudethirte = location.latitude
longitudethirte = location.longitude
print('The geograpical coordinate of 75013 are {}, {}.'.format(latitudethirte, longitudethirte))

address = '75014'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefourte = location.latitude
longitudefourte = location.longitude
print('The geograpical coordinate of 75014 are {}, {}.'.format(latitudefourte, longitudefourte))

address = '75015'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefifte = location.latitude
longitudefifte = location.longitude
print('The geograpical coordinate of 75015 are {}, {}.'.format(latitudefifte, longitudefifte))

address = '75016'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudesixte = location.latitude
longitudesixte = location.longitude
print('The geograpical coordinate of 75016 are {}, {}.'.format(latitudesixte, longitudesixte))

address = '75017'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudesevente = location.latitude
longitudesevente = location.longitude
print('The geograpical coordinate of 75017 are {}, {}.'.format(latitudesevente, longitudesevente))

address = '75018'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeeighte = location.latitude
longitudeeighte = location.longitude
print('The geograpical coordinate of 75018 are {}, {}.'.format(latitudeeighte, longitudeeighte))

address = '75019'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeninete = location.latitude
longitudeninete = location.longitude
print('The geograpical coordinate of 75019 are {}, {}.'.format(latitudeninete, longitudeninete))

address = '75020'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudetwenty = location.latitude
longitudetwenty = location.longitude
print('The geograpical coordinate of 75020 are {}, {}.'.format(latitudetwenty, longitudetwenty))

The geograpical coordinate of 75000 are -16.5467946, -49.2692742.
The geograpical coordinate of 75001 are 48.8635304715812, 2.33893950508195.
The geograpical coordinate of 75002 are 48.8672755917913, 2.34442508383285.
The geograpical coordinate of 75003 are 48.8626136831231, 2.36038090700524.
The geograpical coordinate of 75004 are 48.860845, 2.3529288.
The geograpical coordinate of 75005 are 48.8458521977087, 2.34858912350899.
The geograpical coordinate of 75006 are 48.85353725, 2.34337024122443.
The geograpical coordinate of 75007 are 48.855896700173, 2.31369940048725.
The geograpical coordinate of 75008 are 48.8723737355134, 2.31226306368513.
The geograpical coordinate of 75009 are 48.877464519673, 2.33676866041363.
The geograpical coordinate of 75010 are 48.8792014, 2.3543906.
The geograpical coordinate of 75011 are 48.8587491829173, 2.37782693255991.
The geograpical coordinate of 75012 are 48.8426096368254, 2.38762639345297.
The geograpical coordinate of 75013 are 48.8293667109782

In [191]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) 

# add 75000 as a red circle mark
folium.features.CircleMarker(
    [latitudezero, longitudezero],
    radius=10,
    popup='75000',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75001 as a red circle mark
folium.features.CircleMarker(
    [latitudeone, longitudeone],
    radius=10,
    popup='75001',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75002 as a red circle mark
folium.features.CircleMarker(
    [latitudetwo, longitudetwo],
    radius=10,
    popup='75002',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75003 as a red circle mark
folium.features.CircleMarker(
    [latitudethree, longitudethree],
    radius=10,
    popup='75003',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75004 as a red circle mark
folium.features.CircleMarker(
    [latitudefour, longitudefour],
    radius=10,
    popup='75004',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75005 as a red circle mark
folium.features.CircleMarker(
    [latitudefive, longitudefive],
    radius=10,
    popup='75005',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75006 as a red circle mark
folium.features.CircleMarker(
    [latitudesix, longitudesix],
    radius=10,
    popup='75006',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75007 as a red circle mark
folium.features.CircleMarker(
    [latitudeseven, longitudeseven],
    radius=10,
    popup='75007',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75008 as a red circle mark
folium.features.CircleMarker(
    [latitudeeight, longitudeeight],
    radius=10,
    popup='75008',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75009 as a red circle mark
folium.features.CircleMarker(
    [latitudenine, longitudenine],
    radius=10,
    popup='75009',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75010 as a red circle mark
folium.features.CircleMarker(
    [latitudeten, longitudeten],
    radius=10,
    popup='75010',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75011 as a red circle mark
folium.features.CircleMarker(
    [latitudeeleven, longitudeeleven],
    radius=10,
    popup='75011',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75012 as a red circle mark
folium.features.CircleMarker(
    [latitudetwelve, longitudetwelve],
    radius=10,
    popup='75012',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75013 as a red circle mark
folium.features.CircleMarker(
    [latitudethirte, longitudethirte],
    radius=10,
    popup='75013',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75014 as a red circle mark
folium.features.CircleMarker(
    [latitudefourte, longitudefourte],
    radius=10,
    popup='75014',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75015 as a red circle mark
folium.features.CircleMarker(
    [latitudefifte, longitudefifte],
    radius=10,
    popup='75015',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75016 as a red circle mark
folium.features.CircleMarker(
    [latitudesixte, longitudesixte],
    radius=10,
    popup='75016',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75017 as a red circle mark
folium.features.CircleMarker(
    [latitudesevente, longitudesevente],
    radius=10,
    popup='75017',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75018 as a red circle mark
folium.features.CircleMarker(
    [latitudeeighte, longitudeeighte],
    radius=10,
    popup='75018',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75019 as a red circle mark
folium.features.CircleMarker(
    [latitudeninete, longitudeninete],
    radius=10,
    popup='75019',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75020 as a red circle mark
folium.features.CircleMarker(
    [latitudetwenty, longitudetwenty],
    radius=10,
    popup='75020',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        # Map does not appear if the popup in CircleMarker contains an apostrophe, so I remove apostrophe
        popup=label.replace("'",""),
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map